In [ ]:
from functions import (prep, folding, inbu, LSTM_model)

import numpy as np
import pandas as pd

# import session_info
# session_info.show()

In [ ]:
np.random.seed(42)
filename = "running-example"
flattent_by = 'Orders'
ocel, act_dict, cust_dict = prep.prepare_flat_ocel(filename, flatten_on= flattent_by, filter=prep.act_filter)
print(act_dict)
print(cust_dict)
drops_col_order = ["weight", "price", "Event_ID", 'Products']
ocel = prep.gen_enriched_single_plus_csv(OCEL = ocel,flatted_by = 'Orders',csvname = 'orders_complete', drops_col= drops_col_order)

In [ ]:
ocel =prep.generate_features(ocel)
divisor = np.mean(ocel['Time_Diff'])  # average time between events
divisor2 = np.mean(ocel['Time_Since_Start'])  # average time between current and first events
divisorTR = np.mean(ocel['Remaining_Time'])  # average time instance remaining
print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
print(len(ocel))
ocel_train, ocel_test = folding.folding_train_test(ocel)

defining the features

In [ ]:
act_feat = list(filter(lambda k: k.startswith('Act_') and not k.startswith('Next_Act_'), ocel_train.columns))
target_act_feat = list(filter(lambda k: k.startswith('Next_Act_') and not k.startswith('Act_'), ocel_train.columns))
act_feat.remove('Act_!')
cust_feat = list(filter(lambda k: 'Cust_' in k, ocel_train.columns))
time_feat = ['Time_Diff', 'Time_Since_Start', 'Time_Since_Midnight','Weekday']
other_features = ['Amount_Items','In_Package','Position']
feature_select = act_feat + cust_feat + time_feat + other_features
print(f"Length of act_feat: {len(act_feat)}, Length of cust_feat: {len(cust_feat)}")

## define dimensions of inputs
traces, max_trace_length = prep.gen_traces_and_maxlength_of_trace(ocel)
target_act_length = len(target_act_feat)
number_of_train_cases = len(ocel_train)
num_of_features = len(feature_select)
print(f"Number of train cases: {number_of_train_cases}, Max trace length: {max_trace_length}, Number of features: {num_of_features}")


get the inputs 

In [ ]:

X_train,y_train_a, y_train_t, y_train_tr = inbu.generating_inputs(OCEL=ocel_train,
                                                                  num_of_features=num_of_features,
                                                                  taf=target_act_feat,
                                                                  act=act_feat,
                                                                  custf=cust_feat,
                                                                  divisor_next=divisor,
                                                                  divisor_since=divisor2,
                                                                  divisor_remaining=divisorTR, single= True)

print(f"Shape of X_train: {X_train.shape}")
print(f"This matches the desired shape (number_of_train_cases, max_trace_length, num_of_features): {(number_of_train_cases, max_trace_length, num_of_features)} => {X_train.shape ==(number_of_train_cases, max_trace_length, num_of_features)}")
print(f"Shape of y_train_a: {y_train_a.shape}, this matches the desired shape (number_of_train_cases, target_act_length): {(number_of_train_cases, target_act_length)} => {y_train_a.shape ==(number_of_train_cases, target_act_length)}")
print(f"Shape of y_train_t: {y_train_t.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_t.shape ==(number_of_train_cases, )}")
print(f"Shape of y_train_tr: {y_train_tr.shape}, this matches the desired shape (number_of_train_cases, ): {(number_of_train_cases, )} => {y_train_tr.shape ==(number_of_train_cases, )}")

In [ ]:
history = LSTM_model.LSTM_MODEL(X_train, y_train_a, y_train_t, y_train_tr,filename='orders_filter_single')

In [ ]:
# list all data in history
from matplotlib import pyplot as plt


print(history.history.keys())
# summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()